# [모듈 9.1] 추론 및 결과 분석

이 노트북은 이전 노트북에서 생성한 앤드포인트를 대상으로 실제 추론을 하여, 평가 결과를 얻습니다. 테스트 데이터 세트는 1000개를 사용하여 결과를 확인 합니다.

## 0. 기본 세이지 메이커 정보 및 기본 변수 로딩

In [4]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

%store -r 

## 1. 환경 셋업
- 아래는 사용자 정의의 함수를 로딩 할때마다, 캐시에 있는 것보다는 원본을 로딩함.

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. 테스트 데이터 세트 로딩

In [6]:
import os
test_preproc_dir_artifact_file= os.path.join(test_preproc_dir_artifact, 'test.csv')
test_df = pd.read_csv(test_preproc_dir_artifact_file)
test_df.head()

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
0,0,23901.432507,36201.432507,56,118,0,1,750,3000,98039,...,0,0,0,1,0,0,0,1,0,1
1,0,29095.295736,73695.295736,36,71,0,1,750,2650,95307,...,0,0,0,1,0,0,0,1,0,1
2,0,11232.831826,25032.831826,24,11,1,4,750,3000,98391,...,0,0,1,0,0,0,1,0,1,0
3,0,11125.579356,25125.579356,23,65,0,1,750,3000,91302,...,0,0,1,0,0,0,1,0,1,0
4,0,4529.773913,22329.773913,37,142,0,1,750,3000,90744,...,0,0,1,0,0,0,0,1,1,0


## 3. 추론

#### 추론에 입력될 CST String 형태의 데이터로 변경

In [7]:
def get_payload_list(test_df, label_col):
    '''
    CSV String 행태로 변경
    '''
    s = test_df.drop(columns=[label_col])
    s = s.to_csv(header=None, index=None)
    payload_list = s.splitlines()
    return payload_list

payload_list = get_payload_list(test_df, label_col='fraud')
y_test = test_df['fraud'].tolist()

print("payload_list: \n", payload_list[0:5]) # 샘플로 5개의 리스트 보여줌.


payload_list: 
 ['23901.43250714285,36201.43250714285,56,118,0,1,750,3000,98039,2012,3,3,1,12300,9,12,3,14,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1', '29095.295735759966,73695.29573575997,36,71,0,1,750,2650,95307,2016,3,1,1,44600,8,8,3,17,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,1', '11232.831825505647,25032.83182550565,24,11,1,4,750,3000,98391,2016,2,0,2,13800,2,11,0,21,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0', '11125.579356093212,25125.579356093214,23,65,0,1,750,3000,91302,2015,2,0,0,14000,8,7,2,7,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0', '4529.773913037943,22329.773913037934,37,142,0,1,750,3000,90744,2008,2,0,1,17800,4,2,1,13,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0']


### 추론 실행
- 앤드포인트가 InServie로 유효하면 추론을 하여, 확률값을 제공함.

In [8]:
%%time

import numpy as np
import boto3
sagemaker_boto_client = boto3.client('sagemaker')

from sagemaker.deserializers import CSVDeserializer
csv_deserializer = CSVDeserializer(accept='text/csv') # 디폴트가 accept가 'text/csv' 이지만 직관적 이유로 기술함.

from src.p_utils import is_available_endpoint, get_predictor, get_payload, predict, delete_endpoint

if is_available_endpoint(all_pipeline_endpoint_name, verbose=False):
    pipeline_predictor = get_predictor(all_pipeline_endpoint_name, sagemaker_session, csv_deserializer)
    pred_prob_list = []
    payload_list = get_payload_list(test_df, label_col='fraud')
    for payload in payload_list:
        pred_prob = predict(pipeline_predictor, payload)
        pred_prob_list.append(float(pred_prob[0]))
    print("pred_prob_list: \n", pred_prob_list[0:5])
else:
    print("Wait for creating an endpoint")

pred_prob_list: 
 [0.10092484205961227, 0.08251918852329254, 0.4293206036090851, 0.23539546132087708, 0.669212281703949]
CPU times: user 2.52 s, sys: 36.2 ms, total: 2.56 s
Wall time: 8.99 s


### 예측의 확률값을 0, 1로 변화하여 리포팅
- threshold 값이 보다 작으면 0, 크면 1로 할당한후에, Confustion Matrix로 보여줌

In [9]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

threshold = 0.5
predictions = [1 if e >= threshold else 0 for e in pred_prob_list ] 
    
print(f"{classification_report(y_true=y_test, y_pred = predictions)}")
cm = confusion_matrix(y_true= y_test, y_pred= predictions)    
print(cm)



              precision    recall  f1-score   support

           0       0.99      0.72      0.83       967
           1       0.09      0.79      0.16        33

    accuracy                           0.72      1000
   macro avg       0.54      0.75      0.50      1000
weighted avg       0.96      0.72      0.81      1000

[[697 270]
 [  7  26]]


## 4. 엔드포인트 삭제
- 과금을 막기 위해서 아래 코드를 꼭 수행 바랍니다.
- is_del_model= True  를 통해서 생성한 모델 및 이전 파이프라인을 통한 엔드포인트 컨피그 와  엔드포인트를 삭제 합니다.

In [10]:
import boto3
boto3_client = boto3.client('sagemaker')
delete_endpoint(boto3_client, all_pipeline_endpoint_name, is_del_model=True )



--- Deleted model: pipelines-eg0revhx2vbq-FraudScratchModel-QxvaJNTtdU
--- Deleted endpoint: all-pipeline-endpoint-545904
--- Deleted endpoint_config: pipelines-eg0revhx2vbq-FraudScratchModel-QxvaJNTtdU-config
